# Review

Limits:
* `limit(e, x, l)` calculates $\displaystyle \lim_{x\to\ell}e$
* `oo` means $\infty$

Differentiation:
* `diff(e, x)` calculates $\displaystyle \frac{\partial e}{\partial x}$
* `diff(e, x, n)` calculates $\displaystyle \frac{\partial^n e}{\partial x^n}$
* `diff(e, x, y, z)` calculates $\displaystyle \frac{\partial^3 e}{\partial x\partial y\partial z}$

Taylor expansion
* `e.series(x, x0, n)` expands an expression $e$ into a Taylor expansion in the variable $x$ about $x_0$, upto degree $n$.

Integration:
* `integrate(e, x)` calculates the indefinite integral $\displaystyle \int e dx$
* `integrate(e, (x, a, b))` calculates the definite integral $\displaystyle \int_a^b e dx$

# Differential Equations

A differential equation is an equation that involves the derivative of an unknown function. The general form is
$$x'(t)=f(t, x(t))$$
whrre the function $x(t)$ is unknown, and we hope to find a function $x(t)$ that satisfy this equation. Such a function is called a **solution**.

An DE is typically used to describe the change of a quantity that depends on the population size.

## Example: the Malthus model

In a short period of time, the population growth is typically proportion to the population sizxe it self. Let $x(t)$ be the population size at time $t$, 
$$x'(t)=r x(t)$$
This is called the Malthus model.

## Solving a DE using sympy

`dsolve(eq)` solve a differential equation `eq` that involves the derivative of an unknown function

How do we construct the derivative of an unknown function?

## Function class
The functions $\sin$, $\cos$, $exp$ etc are all instances of the `Function` class

An unknown function may be constructed using the constructor `Function(name)`

In [2]:
from sympy import *
z= symbols("z")
f=Function("f")
display(f(z))
display(f(1))

display(diff(f(z), z))

f(z)

f(1)

Derivative(f(z), z)

Let's solve the Malthus model

In [3]:
x = Function("x")
t = Symbol("t")
r = Symbol("r", positive=True)

# the right hand side
Malthus = Eq(diff(x(t)), r*x(t))
dsolve(Malthus)


Eq(x(t), C1*exp(r*t))

Note that here $C_1$ is an arbitrary constant, that is, any exponential function with a growth rate $r$ is a solution.  How do we find a specific population size?

## Initial condition
To find a solution, we need a value of the solution at a given $t=t_0$, namely, $x_0=x(t_0)$, this is called an **initial condition**.

The initial condition is given as a python dict, to the **ics** argument. Let us assume that the population size at time $t=0$ is $N$, i.e., $x(0)=N$.

In [4]:
N = Symbol("N")

dsolve(Malthus, ics={x(0):N})


Eq(x(t), N*exp(r*t))

## Example: the logistic model
The solution to the Malthus model is an exponentially growing function. But in a long run, we do not see the population maintaining exponential growth. As population increase, individuals compete for resources, adn thus the growth rate decreases, and eventually pass $0$ and becomes negative. A commonly used model is the logistic model, which assumes that the groeth rate $r$ decreases linearly with the population $x(t)$,
$$ r = a\left(1-\frac{x(t)}{K}\right)$$
resulting in the following logistic equation
$$x'(t)=ax(t)\left(1-\frac{x(t)}K\right),\; a>0$$

## Substitution: subs method
The logistic model can be deriving by substituting the definition of $r$ into the Malthus model. 

In sympy, this is done using the sub method of a sympy expression. For example,
* `Malthus.sub({r: a*(1-x(t)/K})`
* It takes a python dict as input, the keys are the symbols to substitute.

In [5]:
K = Symbol("K")
a = Symbol("r", positive=True)

logistic = Malthus.subs({r:r*(1-x(t)/K)})

display(logistic)
xt=dsolve(logistic, ics={x(0):N})
display(xt)


Eq(Derivative(x(t), t), r*(1 - x(t)/K)*x(t))

Eq(x(t), -K*N*exp(r*t)/((K - N)*(-N*exp(r*t)/(K - N) - 1)))

Lets compute $$\lim_{t\to\infty}x(t)$$
`xt` is an equation
* its left hand side is `xt.args[0]`
* its right hand side is `xt.args[1]`

In [6]:
limit(xt.args[1], t, oo)

K

In [7]:
# Verify that x(t) is a solution.
simplify(logistic.subs({x(t):xt.args[1]}))

True

# Equilibrium

An equilibrium of the differential equation is a constant solution. 

We have seen them in difference equations
* Example: for the difference equation, $x_{t+1}=f(x_t)$, an equilibrium $x_t=x^*$ must satisfy $x^*=f(x^*)$

For a differential equation $x'(t)=f(x(t))$, i.e., $x'=f(x)$, 
* note that here the right hand side does note explicitly depend on the time $t$.
* An equilibrium, i.e., a constant solution $x(t)=x^*$, must satisfy
$(x^*)'=0=f(x^*)$
That is, $$f(x^*)=0$$
is the sufficient and necessary condition that an equilibrium must satisfy.

## Example: the logistic model revisited
$$x'=rx\left(1-\frac{x}{K}\right)$$
The equilibria must satisfy
$$rx\left(1-\frac{x}{K}\right)=0$$
We can solve them using the solve function:

In [8]:
display(logistic)

Eq(Derivative(x(t), t), r*(1 - x(t)/K)*x(t))

In [10]:
# the right hand side fo the equation is in .args[1]
RHS = logistic.args[1]
display(RHS)

solve(RHS, x(t))

r*(1 - x(t)/K)*x(t)

[0, K]

There are two equilibrium: $x(t)=0$, and $x(t)=K$.

# System of Differential Equations

Sometimes, a system state must be described with multiple variables, each one is governed by a differential equation. That is, these systems are defined by a system of differential equationa.

Let us consider the following system that gives a damped oscillation
\begin{align*}
x'(t)&=-rx(t)+y(t)\\
y'(t)&=x(t)-ry(t)
\end{align*}

In [13]:
y=Function("y")
dsolve([Eq(diff(x(t), t), -r*x(t)+y(t)), Eq(diff(y(t),t), -x(t)-r*y(t))], [x(t), y(t)])

[Eq(x(t), C1*exp(-r*t)*sin(t) + C2*exp(-r*t)*cos(t)),
 Eq(y(t), C1*exp(-r*t)*cos(t) - C2*exp(-r*t)*sin(t))]



# Example: The SIRS model
The following model is a modification to the SIR model, incorporating a temporary acquired immunity. Each recovered individual loses immunity at a rate $\rho$ and becomes susceptible.
\begin{align*}
S'&=-\beta SI +\rho R\\
I'&=\beta SI-\gamma I\\
R'&=\gamma I-\rho R
\end{align*}

Unfortunately, this system is not solvable, i.e., it has solutions, but there is no formula. We want to know if the system has a constant solution, i.e., an equiliubrium.


In [18]:
from sympy.abc import beta, gamma, rho, S, I, R
dS = -beta*S*I+rho*R
dI = beta*S*I-gamma*I
dR = gamma*I-rho*R

nonlinsolve([dS, dI, dR], [S, I, R])

{(S, 0, 0), (gamma/beta, R*rho/gamma, R)}